## Preparing and Saving Combined Weather Features as Delta Table

Loads multiple weather datasets, extracts the date, joins them on `deviceId` and `date`, and saves the enriched features to a Delta table for climate modeling.


In [0]:
from pyspark.sql.functions import col, to_date

# Load all weather datasets and extract date
air_temp = spark.table("climaguard.singapore.air_temperature") \
    .withColumn("date", to_date(col("timestamp"))) \
    .withColumnRenamed("value", "temperature") \
    .select("deviceId", "name", "latitude", "longitude", "date", "temperature")

rain = spark.table("climaguard.singapore.rainfall") \
    .withColumn("date", to_date(col("timestamp"))) \
    .withColumnRenamed("value", "rainfall") \
    .select("deviceId", "date", "rainfall")

humidity = spark.table("climaguard.singapore.relative_humidity") \
    .withColumn("date", to_date(col("timestamp"))) \
    .withColumnRenamed("value", "relative_humidity") \
    .select("deviceId", "date", "relative_humidity")

wind_dir = spark.table("climaguard.singapore.wind_direction") \
    .withColumn("date", to_date(col("timestamp"))) \
    .withColumnRenamed("value", "wind_direction") \
    .select("deviceId", "date", "wind_direction")

wind_spd = spark.table("climaguard.singapore.wind_speed") \
    .withColumn("date", to_date(col("timestamp"))) \
    .withColumnRenamed("value", "wind_speed") \
    .select("deviceId", "date", "wind_speed")

# Join all tables on deviceId and date
joined_df = air_temp \
    .join(rain, ["deviceId", "date"], "inner") \
    .join(humidity, ["deviceId", "date"], "inner") \
    .join(wind_dir, ["deviceId", "date"], "inner") \
    .join(wind_spd, ["deviceId", "date"], "inner") \
    .select("deviceId", "name", "latitude", "longitude", "date", 
            "temperature", "rainfall", "relative_humidity", "wind_direction", "wind_speed") \
    .orderBy("date", "deviceId")

# Save the result to Delta
joined_df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .option("mergeSchema", "true") \
    .saveAsTable("climaguard.singapore.climate_weather_features")
